<a href="https://colab.research.google.com/github/obliquesignal/algo-trading-bootcamp/blob/master/5c_Recurrent_Neural_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [ ]:
from keras import models
from keras import layers

In [ ]:
#Import Python Libraries
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
plt.style.use('seaborn')

!pip install yfinance -q
import yfinance as yf

<ipython-input-2-1a3c8ac395cc>:6: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')


# Import and Process Data

In [ ]:
#Import training data
start = datetime(2022, 10, 1)
end = datetime(2023, 2, 28)

train_data = yf.Ticker('AAPL').history(start=start, end=end) #Apple Inc. stock

In [ ]:
#Import test data which is different from training data and leaves a gap
start = datetime(2023, 3, 15)
end = datetime(2023, 5, 29)

test_data = yf.Ticker('AAPL').history(start=start, end=end) #Apple Inc. stock

In [ ]:
#Get only closing prices for training data
train_data = train_data.iloc[:, 3].values

#Get only closing prices for test data
test_data = test_data.iloc[:, 3].values

In [ ]:
from sklearn.preprocessing import StandardScaler

#Standardize training and test data so that they have a mean of zero and standard deviation of 1
scaler = StandardScaler()

train_data = scaler.fit_transform(train_data.reshape(-1,1))
test_data = scaler.fit_transform(test_data.reshape(-1, 1))

In [ ]:
#Create x and y training lists so that y depend on the 20 timesteps/days of x values
xtrain_data = []
ytrain_data = []

for i in range(20, len(train_data)):
  xtrain_data.append(train_data[i-20:i,0])
  ytrain_data.append(train_data[i,0])

In [ ]:
#Create x and y test lists so that y depend on the 20 timesteps/days of x values
xtest_data = []
ytest_data = []

for i in range(20, len(test_data)):
  xtest_data.append(test_data[i-20:i,0])
  ytest_data.append(test_data[i,0])

In [ ]:
#Convert x and y training lists into Numpy arrays and reshape x as (samples, timesteps, features)
xtrain_data = np.array(xtrain_data)
ytrain_data = np.array(ytrain_data)
xtrain_data = np.reshape(xtrain_data, (xtrain_data.shape[0], xtrain_data.shape[1], 1))
xtrain_data.shape

(81, 20, 1)

In [ ]:
#Convert x and y test lists into Numpy arrays and reshape x as (samples, timesteps, features)
xtest_data = np.array(xtest_data)
ytest_data = np.array(ytest_data)
xtest_data = np.reshape(xtest_data, (xtest_data.shape[0], xtest_data.shape[1], 1))
xtest_data.shape

(32, 20, 1)

# Develop Network

In [ ]:
#Stacking recurrent neural network linearly
rnn = models.Sequential()

#Add first LSTM layer with 64 units that accepts the shape of xtrain and returns a sequence to the next layer
rnn.add(layers.LSTM(units=64, return_sequences=True, input_shape=(xtrain_data.shape[1],1)))

#Add regularizing layer that drops half the input features randomly
rnn.add(layers.Dropout(0.5))

#Last LSTM layer since return sequence is false
rnn.add(layers.LSTM(units = 32, return_sequences=False))

#Add regularizing layer that drops half the neurons randomly
rnn.add(layers.Dropout(0.5))

#Add fully connected layer that will output one number
rnn.add(layers.Dense(units=1))

In [ ]:
#Specify the learning parameters: mse is mean square error and mae is mean absolute error
rnn.compile(optimizer='adam', loss='mse', metrics='mae')

In [ ]:
#Summarize network 
rnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 20, 64)            16896     
                                                                 
 dropout (Dropout)           (None, 20, 64)            0         
                                                                 
 lstm_1 (LSTM)               (None, 32)                12416     
                                                                 
 dropout_1 (Dropout)         (None, 32)                0         
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 29,345
Trainable params: 29,345
Non-trainable params: 0
_________________________________________________________________


# Train and Test Network

In [ ]:
#Train network on 30 full cylces or epoch with 32 mini batches of data
rnn.fit(xtrain_data, ytrain_data, epochs=30, batch_size=32)

Epoch 1/30
3/3 [==============================] - 8s 48ms/step - loss: 1.2513 - mae: 0.9564
Epoch 2/30
3/3 [==============================] - 0s 75ms/step - loss: 1.0112 - mae: 0.8656
Epoch 3/30
3/3 [==============================] - 0s 36ms/step - loss: 0.8656 - mae: 0.8015
Epoch 4/30
3/3 [==============================] - 0s 32ms/step - loss: 0.8039 - mae: 0.7597
Epoch 5/30
3/3 [==============================] - 0s 41ms/step - loss: 0.7485 - mae: 0.7482
Epoch 6/30
3/3 [==============================] - 0s 37ms/step - loss: 0.6605 - mae: 0.6994
Epoch 7/30
3/3 [==============================] - 0s 34ms/step - loss: 0.6327 - mae: 0.6795
Epoch 8/30
3/3 [==============================] - 0s 33ms/step - loss: 0.6195 - mae: 0.6589
Epoch 9/30
3/3 [==============================] - 0s 37ms/step - loss: 0.5721 - mae: 0.6259
Epoch 10/30
3/3 [==============================] - 0s 37ms/step - loss: 0.5158 - mae: 0.6069
Epoch 11/30
3/3 [==============================] - 0s 33ms/step - loss: 0.4388 

In [ ]:
#Evaluate network on previously unseen and more recent test data
rnn.evaluate(xtest_data, ytest_data)

1/1 [==============================] - 3s 3s/step - loss: 0.2567 - mae: 0.4364


[0.256708562374115, 0.43638885021209717]